In [128]:
import json
import numpy as np
import scipy.stats

with open('../results/amt_results.json', 'r') as f:
    amtres = json.load(f)
    
with open('../amt/samples/gt.json', 'r') as f:
    samples_gt = json.load(f)
image_ids = samples_gt.keys()
    
gen_res= {}
search_dict = {
    '1': 'gt',
    '2': 'greedy',
    '3': 'beam2',
    '4': 'sampling_050',
    '5': 'sampling_t050',
    '6': 'nucleus_050',
    '7': 'diversebeam2_group2_lambda050'}

search_res_files = [
    '../results/greedy/greedy.json',
    '../results/beam/beam2.json',
    '../results/sampling/sampling_050.json',
    '../results/sampling/sampling_t050.json',
    '../results/nucleus/nucleus_050.json',
    '../results/diversebeam/diversebeam2_group2_lambda050.json']

for num, f in enumerate(search_res_files, 2):
    with open(f, 'r') as fres:
        all_res = json.load(fres)['imgToEval']
        samples = {k:v for k, v in all_res.items() if k in image_ids}        
        gen_res[search_dict[str(num)]] = samples


In [129]:
scores = [
    'Bleu_1',
    'Bleu_2',
    'Bleu_3',
    'Bleu_4',
    'METEOR',
    'ROUGE_L',
    'CIDEr',
    'WMD']

In [177]:
# only one search for now

string = ''

for search in search_dict.values():
    
    if search != 'gt':
        
        for metric in scores:
            aut_score_list = []
            humeval_score_list = []


            for k, v in gen_res[search].items():
                target_score = v[metric]
                aut_score_list.append((str(v['image_id']), target_score))

            for k, v in amtres.items():
                if 'train' in v['image_name']:
                    imgid = v['image_name'].split('_')[-1].lstrip('0')
                else:
                    imgid = str(int(v['image_name'].split('_')[-1]) + 100000)

                rel_av = sum(v['relevance']) / len(v['relevance'])
                corr_av = sum(v['correctness']) / len(v['correctness'])
                comp_av = sum(v['composition']) / len(v['composition'])

                if v['dec'] == search:
                    humeval_score_list.append((imgid, rel_av, corr_av, comp_av))

            automatic = {}
            human = {}
            for (imgid, score) in aut_score_list:
                automatic[imgid] = score
                k = [co for (humid, r, c, co) in humeval_score_list if humid == imgid][0]
                human[imgid] = k

            assert automatic.keys() == human.keys()

            pearson = scipy.stats.pearsonr(list(automatic.values()), list(human.values()))
            spearman = scipy.stats.spearmanr(list(automatic.values()), list(human.values()))
            kendall = scipy.stats.kendalltau(list(automatic.values()), list(human.values()))

            print('Search', search)
            print('Metric', metric)
            if pearson[1] < 0.05:
                print('PEARSON SIGNIFICANT')
            if spearman[1] < 0.05:
                print('SPEARMAN SIGNIFICANT')
            if kendall[1] < 0.05:
                print('KENDALL SIGNIFICANT')
            print(round(pearson[0], 2))
            print(round(spearman[0], 2))
            print(round(kendall[0], 2))
            
            if metric == 'CIDEr':
                string += str(round(pearson[0], 2))
                string += ' & '
                string += str(round(spearman[0], 2))
                string += ' & '
                string += str(round(kendall[0], 2))
                string += ' & '

            print()

    #print(string)
    print('-----')
    print()

            #break

-----

Search greedy
Metric Bleu_1
PEARSON SIGNIFICANT
SPEARMAN SIGNIFICANT
KENDALL SIGNIFICANT
0.41
0.37
0.27

Search greedy
Metric Bleu_2
PEARSON SIGNIFICANT
SPEARMAN SIGNIFICANT
KENDALL SIGNIFICANT
0.39
0.36
0.28

Search greedy
Metric Bleu_3
SPEARMAN SIGNIFICANT
KENDALL SIGNIFICANT
0.29
0.32
0.23

Search greedy
Metric Bleu_4
0.15
0.24
0.18

Search greedy
Metric METEOR
-0.07
-0.07
-0.08

Search greedy
Metric ROUGE_L
PEARSON SIGNIFICANT
0.31
0.29
0.22

Search greedy
Metric CIDEr
0.16
0.27
0.2

Search greedy
Metric WMD
0.04
0.03
0.02

-----

Search beam2
Metric Bleu_1
PEARSON SIGNIFICANT
SPEARMAN SIGNIFICANT
KENDALL SIGNIFICANT
0.42
0.4
0.31

Search beam2
Metric Bleu_2
PEARSON SIGNIFICANT
KENDALL SIGNIFICANT
0.38
0.29
0.23

Search beam2
Metric Bleu_3
PEARSON SIGNIFICANT
0.35
0.25
0.19

Search beam2
Metric Bleu_4
0.23
0.2
0.14

Search beam2
Metric METEOR
0.12
0.09
0.06

Search beam2
Metric ROUGE_L
0.24
0.24
0.18

Search beam2
Metric CIDEr
PEARSON SIGNIFICANT
0.36
0.27
0.21

Search beam2